In [52]:
import pandas as pd
import bz2
import json
import tld
from tld import get_tld

#### Test with DataFrame (DON'T RUN THIS CELL)

In [ ]:
df_quotes_2020 = pd.read_json(file_path_quotes_2020, compression='bz2',lines=True)

### Test with Google Colab Code (with 10k rows)
- load 10'000 rows in order to have a first look at the data. Still need to figure it out if it's possible to load the whole dataset. 

In [53]:
def get_domain(url):
    res = get_tld(url, as_object=True)
    return res.tld

In [63]:
path_to_file = 'data/quotes-2020-domains.json.bz2'
number = 0
df = pd.DataFrame(columns=['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains'])
with bz2.open(path_to_file, 'rb') as s_file:
    for instance in s_file:
        if number == 10000:
            break  
        df = pd.concat([df, pd.read_json(instance,lines=True)], ignore_index=True)    


### Summary Columns
- **quoteID**:      Primary key of the quotation (format: "YYYY-MM-DD-{increasing int:06d}")
- **quotation**:    Text of the longest encountered original form of the quotation
- **speaker**:      Selected most likely speaker
- **qids**:         Wikidata IDs of all aliases that match the selected speaker
- **date**:         Earliest occurrence date of any version of the quotation
- **numOccurences**:Number of time this quotation occurs in the articles
- **probas**:       Array representing the probabilities of each speaker having uttered the quotation
- **urls**:         List of links to the original articles containing the quotation
- **phase**:        Corresponding phase of the data in which the quotation first occurred (A-E)
- **domains**:      Domain of the URL 


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   quoteID         10000 non-null  object        
 1   quotation       10000 non-null  object        
 2   speaker         10000 non-null  object        
 3   qids            10000 non-null  object        
 4   date            10000 non-null  datetime64[ns]
 5   numOccurrences  10000 non-null  object        
 6   probas          10000 non-null  object        
 7   urls            10000 non-null  object        
 8   phase           10000 non-null  object        
 9   domains         10000 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 781.4+ KB


In [81]:
df.head(10)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,domains
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E,[com]
1,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E,[com]
2,2020-02-10-000142,... He (Madhav) also disclosed that the illega...,None,[],2020-02-10 23:45:54,1,"[[None, 0.8926], [Prakash Rai, 0.1074]]",[https://indianexpress.com/article/business/ec...,E,[com]
3,2020-02-15-000053,"... [ I ] f it gets to the floor,",None,[],2020-02-15 14:12:51,2,"[[None, 0.581], [Andy Harris, 0.4191]]",[https://patriotpost.us/opinion/68622-trump-bu...,E,"[us, com]"
4,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E,"[com, com, com, com]"
5,2020-02-27-000223,[ one's ] individual's physical characteristic...,None,[],2020-02-27 08:27:00,1,"[[None, 0.7164], [Prince Charles, 0.2836]]",[https://ukhumanrightsblog.com/2020/02/27/the-...,E,[com]
6,2020-04-15-000176,[ Queen ] can. He can cover.,None,[],2020-04-15 17:30:45,1,"[[None, 0.8956], [Eric DeCosta, 0.0665], [Kenn...",[https://www.pennlive.com/baltimore-ravens/202...,E,[com]
7,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E,[co.uk]
8,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E,[co.uk]
9,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E,[com]


### Try to fill json file with the sample of 10k rows

In [79]:
json_sample = df.to_json()
with open('data/quotes-2020-domain-sample10k.json', 'w') as f:
    json.dump(json_sample, f)  